# A Geo-Located Smart Contract Using Cardano Beam and Marlowe


**Executive Summary**

We present a novel synergy between the Cardano Beam Dapp and the Marlowe Dapp, where Cardano Beam is used to geolocate the role tokens needed for authorization of Marlowe transactions. This enables use cases such as geocaching, scavenger hunts, or token drops that are linked to Marlowe contracts: this melds the geolocation capabilities of Cardano Beam with the contract-logic capabilities of Marlowe.

A video that shows this example being run is available [here](https://youtu.be/DmkYen0eaV0).


**Tools**

- The [Cardano Beam web app](https://cardanobeam.app/) allows geo-placement of Cardano native assets.
- The [Cardano Beam Android app](https://play.google.com/store/apps/details?id=com.cardano_beam_native) allows the retrieval of geo-placed Cardano native assets.
- The [Eternl wallet](https://eternl.io/app/mainnet/welcome) is a light wallet that has Dapp connectivity compatible with Cardano Beam.
- [Ada Handle](https://adahandle.com/) provides an identity for the party allowed to retrieve the geo-placed Cardano native assets.
- [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/) allows one to design a Marlowe Contract.
- [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/blob/bwbush/mainnet/marlowe-cli/ReadMe.md) allows running Marlowe contracts on the blockchain.
- [Cardano CLI](https://github.com/input-output-hk/cardano-node/blob/master/cardano-cli/README.md) supports querying the blockchain.


**Parties**

- The `FAUCET` party creates the Marlowe contract and funds it.
- The `BEAMER` party collects the role tokens for the contract and use them to redeem the funds.


**Scenario**

The `FAUCET` party geographically distributes Cardano native tokens that the `BEAMER` party can redeem for Ada after they have collected the tokens using Cardano Beam and used them to redeem the Ada using a Marlowe smart contract.

1. Using [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/), the `FAUCET` party designs a Marlowe contract that requires three tokens (`Globe`, `Swan`, and `BearGarden`) to authorize the removal of the 75 Ada previously deposited in the contract.
2. Using [Cardano Beam web app](https://cardanobeam.app/), the `FAUCET` party places the three tokens at different geographic locations, with the requirement that they can only be picked up by the holder of the `c.marlowe` [Ada Handle](https://adahandle.com/) token.
3. Using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/blob/bwbush/mainnet/marlowe-cli/ReadMe.md), the `FAUCET` party creates the Marlowe contract and deposits the 75 Ada in it.
4. The `FAUCET` party communicates the contract information to the `BEAMER` party. This can be an on-chain or an off-chain communication of the transaction ID that created the Marlowe contract.
5. The `BEAMER` party travels to the three geographic locations where the tokens were placed and uses their `c.marlowe` token with the [Cardano Beam Android app](https://play.google.com/store/apps/details?id=com.cardano_beam_native) to retrieve them into their [Eternl wallet](https://eternl.io/app/mainnet/welcome).
6. The `BEAMER` party uses their newfound tokens to authorize Marlowe transactions that deposit the 75 Ada in their wallet.

## A Few Preliminaries

Use the `marlowe-cli` compiled for `mainnet`.

In [1]:
git symbolic-ref --short HEAD

bwbush/mainnet


In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0 «mainnet»


Record other versions numbers.

In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

9a9a2c4bc54ba57f02905b9499a4339d775cf68e


Select the network.

In [5]:
MAGIC=(--mainnet)

In [6]:
export CARDANO_NODE_SOCKET_PATH=node.socket

Set the locations of signing keys.

In [7]:
export TREASURY=treasury

Select the faucet.

In [8]:
FAUCET_SKEY="$TREASURY/payment.skey"
FAUCET_ADDR="$(cat $TREASURY/payment.mainnet.address)"
echo "$FAUCET_ADDR"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


Set a few constant parameters.

In [9]:
ADA=1000000

In [10]:
SECOND=1000
MINUTE="$((60 * SECOND))"
HOUR="$((60 * MINUTE))"
DAY="$((24 * HOUR))"

In [11]:
NOW=$(($(date -u +%s) * SECOND))
echo "$NOW"

1665331649000


## The Contract

The contract will receive a deposit of 75 Ada from the `FAUCET` party and disperse 25 Ada each to the holder(s) of the `Globe`, `Swan`, and `BearGarden` role tokens, but only after the dispersement has been authorized by the holder of each role token (via their making a Marlowe `Choice`). We build the contract in [Marlowe Playground](https://marlowe-playground-staging.plutus.aws.iohkdev.io/) and using the `Send to Simulator` and `Download as JSON` buttons download the contract.

![The Beamer Reward contract in Blockly.](beamer.png)

### Create the contract

Set the amount of Ada to be distributed.

In [12]:
DEPOSIT="$((75 * ADA))"

Set the names of the roles required to unlock the funds.

In [13]:
TOKEN_1=Globe
TOKEN_2=Swan
TOKEN_3=BearGarden

Create the contract. This is essentialy the same as the JSON downloaded from the Playground, but it is convenient to insert variable names such as the faucet address and the current time into the contract.

In [14]:
yaml2json << EOI > beamer-1.contract
when:
- case:
    party: {"address" : "$FAUCET_ADDR"}
    deposits: $DEPOSIT
    of_token: {"currency_symbol" : "", "token_name" : ""}
    into_account: {"address" : "$FAUCET_ADDR"}
  then:
    when:
    - case:
        choose_between: [{"from" : 0, "to" : 0}]
        for_choice:
          choice_name: Found $TOKEN_1 Token
          choice_owner: {"role_token" : "$TOKEN_1"}
      then:
        when:
        - case:
            choose_between: [{"from" : 0, "to" : 0}]
            for_choice:
              choice_name: Found $TOKEN_2 Token
              choice_owner: {"role_token" : "$TOKEN_2"}
          then:
            when:
            - case:
                choose_between: [{"from" : 0, "to" : 0}]
                for_choice:
                  choice_name: Found $TOKEN_3 Token
                  choice_owner: {"role_token" : "$TOKEN_3"}
              then:
                pay: $((DEPOSIT / 3))
                token: {"currency_symbol" : "", "token_name" : ""}
                from_account: {"address" : "$FAUCET_ADDR"}
                to: {"party" : {"role_token" : "$TOKEN_1"}}
                then:
                  pay: $((DEPOSIT / 3))
                  token: {"currency_symbol" : "", "token_name" : ""}
                  from_account: {"address" : "$FAUCET_ADDR"}
                  to: {"party" : {"role_token" : "$TOKEN_2"}}
                  then:
                    pay: $((DEPOSIT / 3))
                    token: {"currency_symbol" : "", "token_name" : ""}
                    from_account: {"address" : "$FAUCET_ADDR"}
                    to: {"party" : {"role_token" : "$TOKEN_3"}}
                    then: close
            timeout: $((NOW + 7 * DAY + 3 * SECOND))
            timeout_continuation: close
        timeout: $((NOW + 7 * DAY + 2 * SECOND))
        timeout_continuation: close
    timeout: $((NOW + 7 * DAY + 1 * SECOND))
    timeout_continuation: close
timeout: $((NOW + 7 * DAY))
timeout_continuation: close
EOI
cat beamer-1.contract

{"timeout":1665936449000,"timeout_continuation":"close","when":[{"case":{"deposits":75000000,"into_account":{"address":"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h"},"of_token":{"currency_symbol":"","token_name":""},"party":{"address":"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h"}},"then":{"timeout":1665936450000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Found Globe Token","choice_owner":{"role_token":"Globe"}}},"then":{"timeout":1665936451000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Found Swan Token","choice_owner":{"role_token":"Swan"}}},"then":{"timeout":1665936452000,"timeout_continuation":"close","when":[{"case":{"choose_between":[{"from":0,"to":0}],"for_choice":{"choice_name":"Found BearGarden Token","choice_owner":{"role_token":"BearGarden"}}},"then":{"from_account":{"address":"addr1vy9prvx8ufwutkwxx9cmmuua

Set the initial state so that the contract has sufficient minimum UTxO.

In [15]:
yaml2json << EOI > beamer-1.state
accounts:
- - - address: $FAUCET_ADDR
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1
EOI
cat beamer-1.state

{"accounts":[[[{"address":"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1}


## Select the Role Tokens

The only restriction on role tokens for a Marlowe contract is that they must all have the same Policy ID (or, equivalently, the same Currency Symbol). We have previously minted the three role tokens that we'll be using in this example.

In [16]:
ROLES_CURRENCY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d

The tokens are in an [Eternl wallet](https://eternl.io/app/mainnet/welcome), ready for use.

Note that `cardano-cli query utxo` prints token names in hexadecimal. Here is the correspondence between text and hexadecimal for the three tokens.

In [17]:
for t in "$TOKEN_1" "$TOKEN_2" "$TOKEN_3"
do
  echo "$t = $(echo -n $t | basenc --base16 | tr '[:upper:]' '[:lower:]')"
done

Globe = 476c6f6265
Swan = 5377616e
BearGarden = 4265617247617264656e


### `Globe` Token

![`Globe` token](beamer-globe.png)

### `Swan` Token

![`Swan` token](beamer-swan.png)

### `BearGarden` Token

![`BearGarden` token](beamer-beargarden.png)

## Select the Ada Handle Allowed to Retrieve the Role Tokens

In this scenario, we require that the [`$c.marlowe` Ada Handle](https://handle.me/c.marlowe) be present in the wallet associated with the [Cardano Beam Android app](https://play.google.com/store/apps/details?id=com.cardano_beam_native) in order for the wallet to pick up the geolocated role tokens. Other scenarios differing from this example might use a different, more commonly available token to be required to retrieve the assets.

![`$c.marlowe` Ada Handle](beamer-marlowe.png)

Currently, all [Ada Handle](https://adahandle.com/) tokens use the same policy ID.

In [18]:
HANDLE_POLICY=f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a

The token name for the [`$c.marlowe` Ada Handle](https://handle.me/c.marlowe) handle is simply `c.marlowe`.

In [19]:
HANDLE_NAME=c.marlowe

Recall that `cardano-cli query utxo` prints token names in hexadecimal.

In [20]:
echo "$HANDLE_NAME = $(echo -n $HANDLE_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')"

c.marlowe = 632e6d61726c6f7765


## Geographically Place the Role Tokens

We use the [Cardano Beam web app](https://cardanobeam.app/) to place the three role tokens so that only the `$c.marlowe` Ada handle can retrieve them. This involves entering the policy ID `HANDLE_POLICY` in "Token PolicyID" field of the "2a. Required Token" section and `HANDLE_NAME` in the "Token Name" field there. In the "3. Rewards" section, select the role token to be deposited in the "Asset from wallet" field. Pressing "Submit" will open [Eternl wallet](https://eternl.io/app/mainnet/welcome) for signing the transaction that deposits the role token.

### Place the `Globe` Role Token in Boulder

![Token placement in Boulder](beamer-map.png)
![`Globe` role token in Boulder](beamer-boulder.png)

In [21]:
TOKEN_1_ADDR=addr1wxsh083ywy3mag59ex7klhgzadcznqk4mymmcacap5zd03q752jy3
TOKEN_1_COORD=(regard assume lively)

See that the token does indeed reside at the Cardano Beam contract address.

In [22]:
cardano-cli query utxo "${MAGIC[@]}" --address "$TOKEN_1_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
b5dbc440a57134bf2f54af733d73b37eefd30254c4cd400f688fccf655a7e744     0        5000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.476c6f6265 + TxOutDatumHash ScriptDataInBabbageEra "b68f588e7eefa288509de240b38b66d1265e4f2aa5282c9141d954070751b939"


### Place the `Swan` Role Token in Arvada

![`Swan` role token in Arvada](beamer-arvada.png)

In [23]:
TOKEN_2_ADDR=addr1w9nys0k6zj2lq259wq3fnak726sur6jce9gfpmc67gr8qngpuu2js
TOKEN_2_COORD=(wake union gladiators)

See that the token does indeed reside at the Cardano Beam contract address.

In [24]:
cardano-cli query utxo "${MAGIC[@]}" --address "$TOKEN_2_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7128d1ce9828728c3515ee4cd1c20702b8db7da61546ef5f644231f423921f0b     0        5000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumHash ScriptDataInBabbageEra "8fc63dca90904e66dc92f26c9ebdfca26249956b32ed632d260b4048221b06dd"


### Place the `BearGarden` Role Token in Longmont

![`BearGarden` role token in Longmont](beamer-longmont.png)

In [25]:
TOKEN_3_ADDR=addr1w8cw66gmzkwngck8scrknx3r5e2zm0h89k8g7wyncnmj60gp0dgm3
TOKEN_3_COORD=(belong rising humid)

See that the token does indeed reside at the Cardano Beam contract address.

In [26]:
cardano-cli query utxo "${MAGIC[@]}" --address "$TOKEN_3_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ebdcd3a1eb5511a05807547c830ad93eeb026949678b28763943ec58a260fd50     0        5000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4265617247617264656e + TxOutDatumHash ScriptDataInBabbageEra "89b01c5bc0b3871b68ccaf1cef99b4059b9049632102000a75d9a64f6f81de8b"


## Transaction 1. Create the Marlowe Contract

The `FAUCET` party creates the contract.

In [27]:
marlowe-cli run initialize "${MAGIC[@]}" \
                           --roles-currency "$ROLES_CURRENCY" \
                           --contract-file beamer-1.contract \
                           --state-file    beamer-1.state    \
                           --out-file      beamer-1.marlowe  \
                           --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Submit the transaction.

In [28]:
TX_1=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --marlowe-out-file beamer-1.marlowe \
                             --required-signer "$FAUCET_SKEY" \
                             --change-address "$FAUCET_ADDR" \
                             --out-file /dev/null \
                             --submit 600 \
                             --print-stats \
)
echo "TX_1 = $TX_1"


Fee: Lovelace 213549
Size: 976 / 16384 = 5%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 7dbacd750b478f74ce08e57e36c411913d613f8f45e8366690fe4b4911e61ada


View the UTxO at the Marlowe semantics validator address.

In [29]:
MARLOWE_ADDR=$(jq -r '.tx.marloweValidator.address' beamer-1.marlowe)
echo "$MARLOWE_ADDR"

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


In [30]:
cardano-cli query utxo "${MAGIC[@]}" --address "$MARLOWE_ADDR" | sed -n -e "1,2p;/^$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7dbacd750b478f74ce08e57e36c411913d613f8f45e8366690fe4b4911e61ada     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "67d28c9d40cefbc7f35577c8b4d4f14bd040f9b65e28412666693bbc4761cb6d"


## Transaction 2. Deposit the Funds

The `FAUCET` party deposits the 75 Ada into the contract.

In [31]:
marlowe-cli run prepare --marlowe-file beamer-1.marlowe \
                        --out-file     beamer-2.marlowe \
                        --deposit-account "$FAUCET_ADDR" \
                        --deposit-party "$FAUCET_ADDR" \
                        --deposit-amount "$DEPOSIT" \
                        --invalid-before "$((NOW - 5 * MINUTE))" \
                        --invalid-hereafter "$((NOW + 20 * MINUTE))" \
                        --print-stats


Datum size: 575


Submit the transaction.

In [32]:
TX_2=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_1#1" \
                             --marlowe-in-file  beamer-1.marlowe \
                             --marlowe-out-file beamer-2.marlowe \
                             --required-signer "$FAUCET_SKEY" \
                             --change-address "$FAUCET_ADDR" \
                             --out-file /dev/null \
                             --submit 600 \
                             --print-stats \
)
echo "TX_2 = $TX_2"


Fee: Lovelace 1163551
Size: 14238 / 16384 = 86%
Execution units:
  Memory: 4712588 / 14000000 = 33%
  Steps: 1310258360 / 10000000000 = 13%
TX_2 = 5e2f47ac5b2911b5e260b7dabba9975c4b6b585ee9a28269518fcc840f6db3a3


View the UTxO with the deposit at the Marlowe semantics validator address.

In [33]:
cardano-cli query utxo "${MAGIC[@]}" --address "$MARLOWE_ADDR" | sed -n -e "1,2p;/^$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5e2f47ac5b2911b5e260b7dabba9975c4b6b585ee9a28269518fcc840f6db3a3     1        77000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "3d20e9633986cc087129a00856cfc8cb8ada0bff78192e3edc9c98aa9f3f56a5"


## The `BEAMER` Party Travels to Retrieve the Role Tokens

The `BEAMER` party uses the [Cardano Beam Android app](https://play.google.com/store/apps/details?id=com.cardano_beam_native) to navigate to and retrieve each role token.

|   |   |
|---|---|
| ![Map on Beamer Android app](beamer-android.jpg) | ![Retrieval dialog box on Beamer Android app](beamer-retrieve.jpg) |

In this example, the `BEAMER` party is using the following wallet address.

In [34]:
BEAMER_SKEY="$TREASURY/nami.payment-0.skey"
BEAMER_ADDR="$(cat $TREASURY/nami.payment-0.mainnet.address)"
echo "$BEAMER_ADDR"

addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


Note that they have the `$c.marlowe` handle in their wallet.

In [35]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BEAMER_ADDR" | sed -n -e "1,2p;/$HANDLE_POLICY/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ca61162250045d526f9c0bbcbe0d6311e0bab4bf08b782eba16b5844e33ab555     0        1481480 lovelace + 1 f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a.632e6d61726c6f7765 + TxOutDatumNone


### Retrieve the Tokens by Visiting Each Location

|   |   |   |
|---|---|---|
| ![Retrieved first token](beamer-token-1.jpg) | ![Retrieved third token](beamer-token-3.jpg) | ![Retrieved second token](beamer-token-2.jpg) |

After retrieving the tokens, the `BEAMER` party also has them in their wallet, of course.

In [36]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BEAMER_ADDR" | sed -n -e "1,2p;/$HANDLE_POLICY/p;/$ROLES_CURRENCY/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8f0e18e05c53f02c1a0d69e5825a7b9a6f8d6c2df09a6cf52e01b562224c866f     1        1344720 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.476c6f6265 + TxOutDatumNone
ec2403cff13f8e6cabfa3e96f8dd0384d948dbc1c7239e413eb7aaa22b2ba11a     0        1379200 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4265617247617264656e + TxOutDatumNone
f5c946388e3c8523fe707cfff7e962f7fac5cdc2a946ee3534f2dc4d8a797ca1     0        1344720 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone
f5c946388e3c8523fe707cfff7e962f7fac5cdc2a946ee3534f2dc4d8a797ca1     1        1379200 lovelace + 1 f0ff48bbb7bbe9d59a40f1ce90e9e9d0ff5002ec48f232b49ca0fb9a.632e6d61726c6f7765 + TxOutDatumNone


![Role tokens in wallet](beamer-wallet.png)

## Transaction 3. Prove Ownership of the First Two Role Tokens

In order to avoid exceeding the execution-cost constraints of the Cardano protocol limits, we split our authorization of retrieving funds into two transactions.

In [37]:
LATER="$(($(date -u +%s) * SECOND))"
echo "$LATER"

1665362546000


In [38]:
marlowe-cli run prepare --marlowe-file beamer-2.marlowe \
                        --out-file     beamer-3.marlowe \
                        --choice-name "Found $TOKEN_1 Token" \
                        --choice-number 0 \
                        --choice-party "$TOKEN_1" \
                        --choice-name "Found $TOKEN_2 Token" \
                        --choice-number 0 \
                        --choice-party "$TOKEN_2" \
                        --invalid-before "$((LATER - 5 * MINUTE))" \
                        --invalid-hereafter "$((LATER + 10 * MINUTE))" \
                        --print-stats


Datum size: 509


Submit the transaction.

In [39]:
TX_3=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_2#1" \
                             --marlowe-in-file  beamer-2.marlowe \
                             --marlowe-out-file beamer-3.marlowe \
                             --required-signer "$BEAMER_SKEY" \
                             --change-address "$BEAMER_ADDR" \
                             --out-file /dev/null \
                             --submit 600 \
                             --print-stats \
)
echo "TX_3 = $TX_3"


Fee: Lovelace 1568504
Size: 14357 / 16384 = 87%
Execution units:
  Memory: 9831832 / 14000000 = 70%
  Steps: 2555369637 / 10000000000 = 25%
TX_3 = 036888161252f9324872366d9cf200659c807c7ecbd9c6ce4fdb06df77d2c97e


The contract state records that the two authorizations have been received.

In [40]:
jq '.tx.state' beamer-3.marlowe | json2yaml

accounts:
- - - address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
    - currency_symbol: ''
      token_name: ''
  - 77000000
boundValues: []
choices:
- - choice_name: Found Globe Token
    choice_owner:
      role_token: Globe
  - 0
- - choice_name: Found Swan Token
    choice_owner:
      role_token: Swan
  - 0
minTime: 1665362246000


## Transaction 4. Prove Ownership of the Third Role Token

The authorization with the third role token triggers the payments from the contract and its closure.

In [41]:
LATER="$(($(date -u +%s) * SECOND))"
echo "$LATER"

1665362624000


In [42]:
marlowe-cli run prepare --marlowe-file beamer-3.marlowe \
                        --out-file     beamer-4.marlowe \
                        --choice-name "Found $TOKEN_3 Token" \
                        --choice-number 0 \
                        --choice-party "$TOKEN_3" \
                        --invalid-before "$((LATER - 5 * MINUTE))" \
                        --invalid-hereafter "$((LATER + 10 * MINUTE))" \
                        --print-stats


Datum size: 166
Payment 1
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "Globe"
  Ada: Lovelace {getLovelace = 25000000}
Payment 2
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "Swan"
  Ada: Lovelace {getLovelace = 25000000}
Payment 3
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "BearGarden"
  Ada: Lovelace {getLovelace = 25000000}
Payment 4
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Ada: Lovelace {getLovelace = 2000000}


In [43]:
jq '.tx.contract' beamer-4.marlowe

"close"


Submit the transaction.

In [44]:
TX_4=$(
marlowe-cli run auto-execute "${MAGIC[@]}" \
                             --tx-in-marlowe "$TX_3#1" \
                             --marlowe-in-file  beamer-3.marlowe \
                             --marlowe-out-file beamer-4.marlowe \
                             --required-signer "$BEAMER_SKEY" \
                             --change-address "$BEAMER_ADDR" \
                             --out-file /dev/null \
                             --submit 600 \
                             --print-stats \
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1585628
Size: 13897 / 16384 = 84%
Execution units:
  Memory: 10436882 / 14000000 = 74%
  Steps: 2712660004 / 10000000000 = 27%
TX_4 = 00a0bc7fe3f6e2a11713119efb7dfff9f7aea9c5e7b37ee2c85e41c4618d68c3


The Marlowe semantics validator address no longer has a UTxO for the contract.

In [45]:
cardano-cli query utxo "${MAGIC[@]}" --address "$MARLOWE_ADDR" | sed -n -e "1,2p;/^$TX_4/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The Marlowe payout validator address now holds the funds, waiting for them to be withdrawn.

In [46]:
PAYOUT_ADDR="$(jq -r '.tx.rolesValidator.address' beamer-4.marlowe)"
echo "$PAYOUT_ADDR"

addr1w9yswm4tyqjrmj2xy5glhx9fe7m3n7rwj6fz3qfekly3mucd3rynq


In [47]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PAYOUT_ADDR" | sed -n -e "1,2p;/$TX_4/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
00a0bc7fe3f6e2a11713119efb7dfff9f7aea9c5e7b37ee2c85e41c4618d68c3     2        25000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "89b9a9b5ca08f7f10bdc16d8c8744dd6f7fa60e98bf11cc53eefa5560b440909"
00a0bc7fe3f6e2a11713119efb7dfff9f7aea9c5e7b37ee2c85e41c4618d68c3     3        25000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "ebcfe2e8dfff3b71e921c4ca2ba1ab26445a901b1fe1967a5e34531cc282143e"
00a0bc7fe3f6e2a11713119efb7dfff9f7aea9c5e7b37ee2c85e41c4618d68c3     4        25000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "1645d37b3c5e795995618259b8dc0a357ac92ca047d0913ac3e6cacbb7172ab3"


The `BEAMER` party still has the role tokens in their wallet.

In [48]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BEAMER_ADDR" | sed -n -e "1,2p;/$ROLES_CURRENCY/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
00a0bc7fe3f6e2a11713119efb7dfff9f7aea9c5e7b37ee2c85e41c4618d68c3     5        1172320 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4265617247617264656e + TxOutDatumNone
036888161252f9324872366d9cf200659c807c7ecbd9c6ce4fdb06df77d2c97e     2        1150770 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.476c6f6265 + TxOutDatumNone
036888161252f9324872366d9cf200659c807c7ecbd9c6ce4fdb06df77d2c97e     3        1146460 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone


## Transactions 5-7. Withdraw the Ada from the Payout Address

Now the `BEAMER` party uses their role tokens to withdraw the three payments of 25 Ada from the payout validator.

In [49]:
for t in "$TOKEN_1" "$TOKEN_2" "$TOKEN_3"
do
  marlowe-cli run auto-withdraw "${MAGIC[@]}" \
                                --marlowe-file beamer-1.marlowe \
                                --role-name "$t" \
                                --required-signer "$BEAMER_SKEY" \
                                --change-address "$BEAMER_ADDR" \
                                --out-file /dev/null \
                                --submit 600 \
                                --print-stats
done


Fee: Lovelace 457628
Size: 3142 / 16384 = 19%
Execution units:
  Memory: 1841380 / 14000000 = 13%
  Steps: 509893779 / 10000000000 = 5%
badb8ea25b1183c2710e990e846016e194318e9c26ea8f5b69a0f8925292aa4f

Fee: Lovelace 453478
Size: 3140 / 16384 = 19%
Execution units:
  Memory: 1786580 / 14000000 = 12%
  Steps: 497408858 / 10000000000 = 4%
45f1510d0f395e5661178664f403a8f0b97e65a5b30523f80b341cb42c0ffd1d

Fee: Lovelace 454006
Size: 3152 / 16384 = 19%
Execution units:
  Memory: 1786580 / 14000000 = 12%
  Steps: 497408982 / 10000000000 = 4%
b7c51418b780035d0cfd48b6a6f5bf82f4f12aad551b27567aef4f67dabb046d


They now have the funds in their wallet.

In [50]:
cardano-cli query utxo "${MAGIC[@]}" --address "$BEAMER_ADDR" | sed -n -e "1,2p;/$ROLES_CURRENCY/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
45f1510d0f395e5661178664f403a8f0b97e65a5b30523f80b341cb42c0ffd1d     1        25000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.5377616e + TxOutDatumNone
b7c51418b780035d0cfd48b6a6f5bf82f4f12aad551b27567aef4f67dabb046d     1        25000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4265617247617264656e + TxOutDatumNone
badb8ea25b1183c2710e990e846016e194318e9c26ea8f5b69a0f8925292aa4f     1        25000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.476c6f6265 + TxOutDatumNone
